# Imports


* In this assignment you will be using the entire transformer architecture for a translation task.
* we will just be using one encoder layer and one decoder layer
* You can copy the encoder and decoder modules from the previous assignments. You are going to translate a few sentences from **English to Tamil**
  * Source language: English
  * Target language: Tamil

* You may experiment with a target language of your choice for checking the impelementation. (You may use google translate for that)

* We need to install torchdata and torchtext (which take about 3 minutes to finish installing) for tokenizing the text.
* We already defined useful functions for the tokenization of texts




In [1]:
!pip install torchdata==0.6.0 # to be compatible with torch 2.0
!pip install portalocker==2.0.0
!pip install -U torchtext==0.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

* Let's import all required libraries

In [2]:
import torch
from torch import Tensor

import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn.functional import one_hot

import torch.optim as optim

#text lib
import torchtext

# tokenizer
from torchtext.data.utils import get_tokenizer

#build vocabulary
from torchtext.vocab import vocab
from torchtext.vocab import build_vocab_from_iterator

# get input_ids (numericalization)
from torchtext.transforms import VocabTransform, LabelToIndex

# get embeddings
from torch.nn import Embedding

from  pprint import pprint
from yaml import safe_load
import copy
import numpy as np
import requests
import math

# Preparing Data

* Source and target text

In [117]:
src_text = """The most famous ruler of ancient India was Emperor Ashoka.
It was during his period that Buddhism spread to different parts of Asia.
Ashoka gave up war after seeing many people grieving death after the Kalinga war.
He embraced Buddhism and then devoted his life to spread the message of peace and dharma.
His service for the cause of public good was exemplary.
He was the first ruler to give up war after victory.
He was the first to build hospitals for animals.
He was the first to lay roads."""

In [118]:
tar_text = """பண்டைய இந்திய அரசர்களில் பேரும் புகழும் பெற்ற அரசர் அசோகர் ஆவார்.
இவரது ஆட்சியில் தான் புத்த மதம் ஆசியாவின் பல்வேறு பகுதிகளுக்குப் பரவியது.
கலிங்கப் போருக்குப் பின் பல உயிர்கள் மடிவதைக் கண்டு வருந்தி, போர் தொடுப்பதைக் கைவிட்டார்.
அதற்குப் பிறகு புத்த சமயத்தைத் தழுவி, அமைதியையும் அறத்தையும் பரப்புவதற்காகத் தன் வாழ்வையே அர்ப்பணித்தார்.
பொதுமக்களுக்கு அவர் ஆற்றிய சேவை முன் மாதிரியாக விளங்கியது.
வெற்றிக்குப் பின் போரைத் துறந்த முதல் அரசர் அசோகர்தான்.
உலகிலேயே முதன்முதலாக விலங்குகளுக்கும் தனியே மருத்துவமனை அமைத்துத் தந்தவரும் அசோகரே ஆவார்.
 இன்றும் அவர் உருவாக்கிய சாலைகளை நாம் பயன்படுத்திக்கொண்டு இருக்கிறோம்."""

* Tokenize and build vocabulary using a simple tokenization algorithm

In [119]:
# do not edit this cell
def seq_len(seq):
  return len(seq.strip('').split(' '))

# check the maximum length of the src and target seq to decide the context length of encdoer and decoder
src_raw_seq = src_text.strip('').split('\n')
src_max_seq_len =max(list(map(seq_len,src_raw_seq)))
print('Source max_seq_length:  ',src_max_seq_len)


tar_raw_seq = tar_text.strip('').split('\n')
tar_max_seq_len =max(list(map(seq_len,tar_raw_seq)))
print('Target max_seq_length: ',tar_max_seq_len)

Source max_seq_length:   16
Target max_seq_length:  11


* We encourage you to go through the code given below to understand the typical functionalities of Tokenizer object (If you want, you can skip)

In [120]:
# do not edit this cell
class Tokenizer(object):

  def __init__(self,text):
    self.text = text
    self.word_tokenizer = self.word_tokenizer
    self.vocab_size = None
    self.vocab = None

  @staticmethod
  def word_tokenizer(seq):
    return seq.strip('').split(' ')

  def get_tokens(self):
    for sentence in self.text.strip().split('\n'):
      yield self.word_tokenizer(sentence)

  def build_vocab(self):
    self.vocab = build_vocab_from_iterator(self.get_tokens(),
                                  min_freq=1,specials=['<pad>','<start>','<end>','<unk>'])
    self.vocab.set_default_index(self.vocab['<unk>']) # index of OOV
    self.vocab_size = len(self.vocab)
    return self.vocab

  def encode(self,sentence):
    v = self.build_vocab()
    vt = VocabTransform(v)
    token_ids = vt(self.word_tokenizer(sentence))
    # add special tokens
    token_ids.insert(0,v.vocab.get_stoi()['<start>'])
    token_ids.append(v.vocab.get_stoi()['<end>']) # <end>:2
    return torch.tensor(token_ids,dtype=torch.int64)

  def decode(self,ids):
    v = self.build_vocab()
    list_ids = ids.tolist()
    tokens = [v.vocab.get_itos()[id] for id in list_ids]
    return ' '.join(tokens)

  def encode_batch(self,batch_size,max_seq_len):
    batch_data = torch.zeros(size=(batch_size,max_seq_len+2)) # +2 for special tokens
    for i,sentence in enumerate(self.text.strip('').split('\n')):
      token_ids = self.encode(sentence)
      batch_data[i,0:len(token_ids)] = token_ids
    return batch_data.type(dtype=torch.int64)



* It is always go to check the implementation

In [121]:
batch_size = 8

In [122]:
# you can play with this
src_tokenizer = Tokenizer(src_text)
print(src_tokenizer.encode('The most famous ruler of ancient India was Emperor Ashoka.'))
print(src_tokenizer.encode_batch(batch_size,src_max_seq_len))

tensor([ 1, 27, 49, 39, 15,  8, 28, 24,  5, 22, 20,  2])
tensor([[ 1, 27, 49, 39, 15,  8, 28, 24,  5, 22, 20,  2,  0,  0,  0,  0,  0,  0],
        [ 1, 25,  5, 36, 14, 53, 58, 11, 16,  6, 35, 50,  8, 21,  2,  0,  0,  0],
        [ 1, 19, 40, 17, 18,  9, 56, 47, 52, 43, 32,  9,  4, 26, 61,  2,  0,  0],
        [ 1,  7, 37, 11, 12, 59, 33, 14, 46,  6, 16,  4, 48,  8, 51, 12, 34,  2],
        [ 1, 23, 57, 13,  4, 31,  8, 54, 42,  5, 38,  2,  0,  0,  0,  0,  0,  0],
        [ 1,  7,  5,  4, 10, 15,  6, 41, 17, 18,  9, 60,  2,  0,  0,  0,  0,  0],
        [ 1,  7,  5,  4, 10,  6, 30, 44, 13, 29,  2,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  7,  5,  4, 10,  6, 45, 55,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0]])


In [123]:
# you can play with this
tar_tokenizer = Tokenizer(tar_text)
print(tar_tokenizer.encode('பண்டைய இந்திய அரசர்களில் பேரும் புகழும் பெற்ற அரசர் அசோகர் ஆவார்.'))
print(tar_tokenizer.encode_batch(batch_size,tar_max_seq_len))

tensor([ 1, 44, 22, 16, 53, 51, 52,  4, 11,  6,  2])
tensor([[ 1, 44, 22, 16, 53, 51, 52,  4, 11,  6,  2,  0,  0],
        [ 1, 25, 20, 39,  8, 59, 19, 49, 43, 47,  2,  0,  0],
        [ 1, 30, 55,  7, 48, 26, 58, 29, 65, 57, 41, 31,  2],
        [ 1, 13, 50,  8, 32, 38, 14, 18, 46, 37, 66, 17,  2],
        [ 1, 54,  5, 21, 34, 64, 61, 68,  2,  0,  0,  0,  0],
        [ 1, 69,  7, 56, 40, 63,  4, 12,  2,  0,  0,  0,  0],
        [ 1, 28, 62, 67, 36, 60, 15, 35, 10,  6,  2,  0,  0],
        [ 1,  9, 23,  5, 27, 33, 42, 45, 24,  2,  0,  0,  0]])


* Let's load the token ids of the words in the sentences of source and target languages

In [124]:
# do not edit this cell
x = src_tokenizer.encode_batch(batch_size,src_max_seq_len)
y = tar_tokenizer.encode_batch(batch_size,tar_max_seq_len)

* we have appended zeros to sentences that are shorter than max-seq-len
* We have to ignore computing loss over those padded tokens
* You have to take care of that in the cell below

In [125]:
# your code goes here
label = y[:, 1:]

* Define the context lengths for encoder and decoder

In [126]:
# do not edit this cell
enc_ctxt_len = src_max_seq_len+2
dec_ctxt_len = tar_max_seq_len+2

# Load configuration file

In [127]:
# do not edit this cell
config_url = "https://raw.githubusercontent.com/Arunprakash-A/LLM-from-scratch-PyTorch/main/config_files/enc_config.yml"
response = requests.get(config_url)
config = response.content.decode("utf-8")
config = safe_load(config)
pprint(config)

{'input': {'batch_size': 10, 'embed_dim': 32, 'seq_len': 8, 'vocab_size': 10},
 'model': {'d_ff': 128,
           'd_model': 32,
           'dk': 4,
           'dq': 4,
           'dv': 4,
           'n_heads': 8,
           'n_layers': 6}}


In [128]:
# do not edit this cell
src_vocab_size =src_tokenizer.vocab_size
batch_size = x.shape[0]
embed_dim = config['input']['embed_dim']

In [129]:
# do not edit this cell
dq = torch.tensor(config['model']['dq'])
dk = torch.tensor(config['model']['dk'])
dv = torch.tensor(config['model']['dv'])
dmodel = embed_dim
heads = torch.tensor(config['model']['n_heads'])
d_ff = config['model']['d_ff']

In [130]:
# do not edit this cell
config_url = "https://raw.githubusercontent.com/Arunprakash-A/LLM-from-scratch-PyTorch/main/config_files/dec_config.yml"
response = requests.get(config_url)
config = response.content.decode("utf-8")
config = safe_load(config)
pprint(config)

{'input': {'batch_size': 10, 'embed_dim': 32, 'seq_len': 8, 'vocab_size': 12},
 'model': {'d_ff': 128,
           'd_model': 32,
           'dk': 4,
           'dq': 4,
           'dv': 4,
           'n_heads': 8,
           'n_layers': 6}}


In [131]:
# do not edit this cell
tar_vocab_size = tar_tokenizer.vocab_size

# Positional Embedding

 * You may take the code directly from any source.

In [155]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len = 500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """ x: Tensor, shape [batch_size, seq_len, embedding_dim] """
        x = x + self.pe[:, :x.size(1)]
        return x

In [156]:
class Embed(nn.Module):

    def __init__(self, vocab_size, embed_dim):
        super(Embed, self).__init__()
        torch.manual_seed(70)
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.dropout = nn.Dropout(p = 0.1)

    def forward(self, x):
        out = self.embed(x)
        # out = self.dropout(out)
        return out

# Encoder

 * You can copy paste the required code from the previous assignments

In [157]:
class MHA(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, heads):
        super(MHA, self).__init__()
        self.heads = heads
        self.dq = dq
        self.dk = dk
        self.dv = dv
        self.dmodel = dmodel
        self.dropout = nn.Dropout(p = 0.1)

        self.WQ = nn.Parameter(self.init_param(dmodel, heads * dq, seed = 43))
        self.WK = nn.Parameter(self.init_param(dmodel, heads * dk, seed = 44))
        self.WV = nn.Parameter(self.init_param(dmodel, heads * dv, seed = 45))
        self.WO = nn.Parameter(self.init_param(heads * dv, dmodel, seed = 46))
        self.scaling_factor = math.sqrt(dk)

    def init_param(self, *size, seed):
        torch.manual_seed(seed)
        return torch.randn(size)

    def forward(self, H = None):
        BS, T, _ = H.shape

        Q = torch.matmul(H, self.WQ.T)
        K = torch.matmul(H, self.WK.T)
        V = torch.matmul(H, self.WV.T)

        Q = Q.view(BS, T, self.heads, self.dq).transpose(1, 2)
        K = K.view(BS, T, self.heads, self.dk).transpose(1, 2)
        V = V.view(BS, T, self.heads, self.dv).transpose(1, 2)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scaling_factor
        attention = F.softmax(attention_scores, dim = -1)
        # attention = self.dropout(attention)

        out = torch.matmul(attention, V)
        out = out.transpose(1, 2).contiguous().view(BS, T, -1)
        out = torch.matmul(out, self.WO.T)
        return out


class FFN(nn.Module):

    def __init__(self, dmodel, d_ff):
        super(FFN, self).__init__()
        torch.manual_seed(47)
        self.linear1 = nn.Linear(dmodel, d_ff)
        self.dropout = nn.Dropout(p = 0.1)
        torch.manual_seed(48)
        self.linear2 = nn.Linear(d_ff, dmodel)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.kaiming_normal_(self.linear1.weight, mode = 'fan_out', nonlinearity = 'relu')
        nn.init.zeros_(self.linear1.bias)
        nn.init.kaiming_normal_(self.linear2.weight, mode = 'fan_out', nonlinearity = 'relu')
        nn.init.zeros_(self.linear2.bias)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        # x = self.dropout(x)
        out = self.linear2(x)
        return out


class Prediction(nn.Module):

    def __init__(self, dmodel, vocab_size):
        super(OutputLayer, self).__init__()
        torch.manual_seed(49)
        self.linear = nn.Linear(dmodel, vocab_size)
        self.dropout = nn.Dropout(p = 0.1)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.zeros_(self.linear.bias)

    def forward(self, representations):
        out = self.linear(representations)
        # out = self.dropout(out)
        return out


class EncoderLayer(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, d_ff, heads):
        super(EncoderLayer, self).__init__()
        self.mha = MHA(dmodel, dq, dk, dv, heads)
        self.layer_norm_mha = nn.LayerNorm(dmodel)
        self.ffn = FFN(dmodel, d_ff)
        self.layer_norm_ffn = nn.LayerNorm(dmodel)

    def forward(self, x):
        # Pre Layer Norm
        x_norm = self.layer_norm_mha(x)
        attention_output = self.mha(x_norm)
        x = x + attention_output

        x_norm = self.layer_norm_ffn(x)
        ffn_output = self.ffn(x_norm)
        out = x + ffn_output
        return out

class Encoder(nn.Module):

    def __init__(self, vocab_size, embed_dim, dq, dk, dv, d_ff, heads, num_layers = 1, max_len = 500):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        torch.nn.init.xavier_uniform_(self.embedding.weight)
        self.pos_encoder = PositionalEncoding(embed_dim, max_len)
        self.encoder_layers = nn.ModuleList([EncoderLayer(embed_dim, dq, dk, dv, d_ff, heads) for _ in range(num_layers)])
        self.final_layer_norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        for layer in self.encoder_layers :
            x = layer(x)
        x = self.final_layer_norm(x)
        return x

# Decoder

In [158]:
class MHCA(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, heads):
        super(MHCA, self).__init__()
        self.heads = heads
        self.dq = dq
        self.dk = dk
        self.dv = dv
        self.dmodel = dmodel
        self.dropout = nn.Dropout(p = 0.1)

        self.WQ = nn.Parameter(self.init_param(dmodel, heads * dq, seed = 43))
        self.WK = nn.Parameter(self.init_param(dmodel, heads * dk, seed = 44))
        self.WV = nn.Parameter(self.init_param(dmodel, heads * dv, seed = 45))
        self.WO = nn.Parameter(self.init_param(heads * dv, dmodel, seed = 46))
        self.scaling_factor = math.sqrt(dk)

    def init_param(self, *size, seed):
        torch.manual_seed(seed)
        return torch.randn(size)

    def forward(self, decoder_input, encoder_output):
        BS, T_dec, _ = decoder_input.shape
        _, T_enc, _ = encoder_output.shape

        Q = torch.matmul(decoder_input, self.WQ.T).view(BS, T_dec, self.heads, -1).transpose(1, 2)
        K = torch.matmul(encoder_output, self.WK.T).view(BS, T_enc, self.heads, -1).transpose(1, 2)
        V = torch.matmul(encoder_output, self.WV.T).view(BS, T_enc, self.heads, -1).transpose(1, 2)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scaling_factor
        attention = F.softmax(attention_scores, dim = -1)
        # attention = self.dropout(attention)

        out = torch.matmul(attention, V).transpose(1, 2).contiguous().view(BS, T_dec, -1)
        out = torch.matmul(out, self.WO.T)
        return out


class MHMA(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, heads, mask = None):
        super(MHMA, self).__init__()
        self.heads = heads
        self.dq = dq
        self.dk = dk
        self.dv = dv
        self.dmodel = dmodel
        self.mask = mask
        self.dropout = nn.Dropout(p = 0.1)

        self.WQ = nn.Parameter(self.init_param(heads * dq, dmodel, seed = 43))
        self.WK = nn.Parameter(self.init_param(heads * dk, dmodel, seed = 44))
        self.WV = nn.Parameter(self.init_param(heads * dv, dmodel, seed = 45))
        self.WO = nn.Parameter(self.init_param(dmodel, heads * dv, seed = 46))
        self.scaling_factor = math.sqrt(dk)

    def init_param(self, *size, seed):
        torch.manual_seed(seed)
        return torch.randn(size)

    def forward(self, x, mask=None):
        BS, T, _ = x.shape
        Q = torch.matmul(x, self.WQ.T).view(BS, T, self.heads, -1).transpose(1, 2)
        K = torch.matmul(x, self.WK.T).view(BS, T, self.heads, -1).transpose(1, 2)
        V = torch.matmul(x, self.WV.T).view(BS, T, self.heads, -1).transpose(1, 2)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scaling_factor

        if mask is None:
            mask = torch.triu(torch.ones((T, T), device = x.device, dtype = torch.bool), diagonal = 1)
            mask = mask[None, None, :, :].expand(BS, self.heads, T, T)
            attention_scores = attention_scores.masked_fill(mask, float('-inf'))
        else :
            attention_scores = attention_scores + mask

        attention = F.softmax(attention_scores, dim=-1)
        # attention = self.dropout(attention)

        out = torch.matmul(attention, V).transpose(1, 2).contiguous().view(BS, T, -1)
        out = torch.matmul(out, self.WO.T)
        return out


class FFN(nn.Module):

    def __init__(self, dmodel, d_ff):
        super(FFN, self).__init__()
        torch.manual_seed(47)
        self.linear1 = nn.Linear(dmodel, d_ff)
        self.dropout = nn.Dropout(p = 0.1)
        torch.manual_seed(48)
        self.linear2 = nn.Linear(d_ff, dmodel)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.kaiming_normal_(self.linear1.weight, mode='fan_out', nonlinearity='relu')
        nn.init.zeros_(self.linear1.bias)
        nn.init.kaiming_normal_(self.linear2.weight, mode='fan_out', nonlinearity='relu')
        nn.init.zeros_(self.linear2.bias)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        # x = self.dropout(x)
        out = self.linear2(x)
        return out


class OutputLayer(nn.Module):

    def __init__(self, dmodel, vocab_size):
        super(OutputLayer, self).__init__()
        torch.manual_seed(49)
        self.linear = nn.Linear(dmodel, vocab_size)
        self.dropout = nn.Dropout(p = 0.1)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.zeros_(self.linear.bias)

    def forward(self, representations):
        out = self.linear(representations)
        # out = self.dropout(out)
        return out


class DecoderLayer(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, d_ff, heads, mask = None):
        super(DecoderLayer, self).__init__()
        self.mhma = MHMA(dmodel, dq, dk, dv, heads, mask=mask)
        self.mhca = MHCA(dmodel, dq, dk, dv, heads)
        self.ffn = FFN(dmodel, d_ff)

        self.pre_ln_mhma = nn.LayerNorm(dmodel)
        self.pre_ln_mhca = nn.LayerNorm(dmodel)
        self.pre_ln_ffn = nn.LayerNorm(dmodel)

    def forward(self, dec_input, enc_output, self_attn_mask = None):
        # Pre-Layer Normalization for all layers
        norm_mhma_in = self.pre_ln_mhma(dec_input)
        mhma_out = self.mhma(norm_mhma_in, self_attn_mask) + dec_input

        norm_mhca_in = self.pre_ln_mhca(mhma_out)
        mhca_out = self.mhca(norm_mhca_in, enc_output) + mhma_out

        norm_ffn_in = self.pre_ln_ffn(mhca_out)
        ffn_out = self.ffn(norm_ffn_in) + mhca_out
        return ffn_out


class Decoder(nn.Module):

    def __init__(self, vocab_size, dmodel, dq, dk, dv, d_ff, heads, mask, num_layers = 1, max_len = 500):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, dmodel)
        self.pos_encoder = PositionalEncoding(dmodel, max_len)
        self.dec_layers = nn.ModuleList([DecoderLayer(dmodel, dq, dk, dv, d_ff, heads, mask) for _ in range(num_layers)])
        self.final_layer_norm = nn.LayerNorm(dmodel)
        self.output_linear = nn.Linear(dmodel, vocab_size)  # Project back to vocab size

    def forward(self, enc_output, tar_token_ids):
        tar_embed = self.embedding(tar_token_ids)
        tar_embed = self.pos_encoder(tar_embed)
        dec_output = tar_embed
        for layer in self.dec_layers:
            dec_output = layer(dec_output, enc_output)
        dec_output = self.final_layer_norm(dec_output)
        logits = self.output_linear(dec_output)
        return logits

# Generate target mask

  * We will be passing the causal mask for the decoder layer as one of its arguments

In [159]:
mask = (torch.triu(torch.ones(dec_ctxt_len,dec_ctxt_len)) == 1).transpose(0,1)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
print(mask)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


# Transformer

In [167]:
class Transformer(nn.Module):

  def __init__(self, src_vocab_size, tar_vocab_size, src_seq_len, tar_seq_len, dmodel, dq, dk, dv, d_ff, heads, target_mask, num_layers = 1):
        super(Transformer,self).__init__()
        self.src_embeddings = nn.Embedding(src_vocab_size, embed_dim)
        self.tar_embeddings = nn.Embedding(tar_vocab_size, embed_dim)
        self.pos_embeddings = PositionalEncoding(dmodel)
        self.encoder = Encoder(src_vocab_size, dmodel, dq, dk, dv, d_ff, heads, num_layers)
        self.decoder = Decoder(tar_vocab_size, dmodel, dq, dk, dv, d_ff, heads, target_mask, num_layers)

  def forward(self, src_token_ids, tar_token_ids):
        encoder_out = self.encoder(src_token_ids) # encoder output
        decoder_out = self.decoder(encoder_out, tar_token_ids)  # decoder output, using the encoder output and target token IDs
        return decoder_out

In [168]:
model = Transformer(src_vocab_size, tar_vocab_size, enc_ctxt_len, dec_ctxt_len, dmodel, dq, dk, dv, d_ff, heads, mask)

In [169]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
#optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [170]:
def train(src_token_ids, tar_token_ids, labels, epochs = 1000):

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        # Exclude the <end> token for decoder input
        decoder_input = tar_token_ids[:, :-1]
        targets = labels

        # Forward pass
        output = model(src_token_ids, decoder_input)
        # Assuming your model's output is [batch_size, seq_length, vocab_size]

        # Reshape for CrossEntropyLoss, if your criterion expects [batch_size * seq_length, vocab_size]
        output = output.reshape(-1, output.shape[-1])
        targets = targets.reshape(-1)

        # Compute loss
        loss = criterion(output, targets)
        # Backward and optimize
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0 :
            print(f'Epoch {epoch}, Loss: {loss.item()}')

In [171]:
train(x, y, label, 1000)

Epoch 0, Loss: 4.5233283042907715
Epoch 100, Loss: 1.205599308013916
Epoch 200, Loss: 0.17830687761306763
Epoch 300, Loss: 0.059763211756944656
Epoch 400, Loss: 0.030649857595562935
Epoch 500, Loss: 0.018908249214291573
Epoch 600, Loss: 0.0129126301035285
Epoch 700, Loss: 0.009422008879482746
Epoch 800, Loss: 0.007192558143287897
Epoch 900, Loss: 0.0056755393743515015


## Run the model AutoRegressively

In [172]:
@torch.inference_mode()
def inference(model, src_token_ids, src_vocab_size, tar_vocab_size, max_len = 50):

      src_token_ids = src_token_ids.long()
      # Initialize with the <start> token
      tar_token_ids = torch.tensor([[src_tokenizer.vocab['<start>']]], device=src_token_ids.device)

      for _ in range(max_len):
          output = model(src_token_ids, tar_token_ids)
          next_token_logits = output[:, -1, :]  # Last token's logits
          next_token_id = next_token_logits.argmax(dim=-1).unsqueeze(-1)

          # Append predicted token
          tar_token_ids = torch.cat([tar_token_ids, next_token_id], dim=-1)

          # Break if <end> token is generated
          if next_token_id.item() == tar_tokenizer.vocab['<end>']:
              break

      return tar_token_ids[:, 1:]  # Remove <start> token for the output

* Modify the code below to suit your implementation
* Display the original and translated sentence (with all the spcial tokens)
* Note that, the second half of the second sentence is poorly translated
*  Same goes for 3rd and 4th sentence
* All other sentences are properly translated

In [173]:
for i, token_ids in enumerate(x[:]):

      # Ensure token_ids is in correct format for src_tokenizer.decode
      original_sentence = src_tokenizer.decode(token_ids if isinstance(token_ids, torch.Tensor) else token_ids)
      print(f"Original [{i+1}]: {original_sentence}")

      # Ensure token_ids is a tensor for model inference
      token_ids_tensor = torch.tensor(token_ids, dtype=torch.long) if not isinstance(token_ids, torch.Tensor) else token_ids

      # Run inference with the tensor
      translated_token_ids = inference(model, token_ids_tensor.unsqueeze(0), src_vocab_size, tar_vocab_size)

      # Prepare translated_token_ids for tar_tokenizer.decode
      if isinstance(translated_token_ids, torch.Tensor):
          translated_list_ids = translated_token_ids.squeeze()
      else:
          translated_list_ids = translated_token_ids

      # Decode the translated sequence
      translated_sentence = tar_tokenizer.decode(translated_list_ids)
      print(f"Translated [{i+1}]: {translated_sentence}\n")

Original [1]: <start> The most famous ruler of ancient India was Emperor Ashoka. <end> <pad> <pad> <pad> <pad> <pad> <pad>
Translated [1]: பண்டைய இந்திய அரசர்களில் பேரும் புகழும் பெற்ற அரசர் அசோகர் ஆவார். <end>

Original [2]: <start> It was during his period that Buddhism spread to different parts of Asia. <end> <pad> <pad> <pad>
Translated [2]: இவரது ஆட்சியில் தான் புத்த மதம் ஆசியாவின் பல்வேறு பகுதிகளுக்குப் பரவியது. <end>

Original [3]: <start> Ashoka gave up war after seeing many people grieving death after the Kalinga war. <end> <pad> <pad>
Translated [3]: கலிங்கப் போருக்குப் பின் பல உயிர்கள் மடிவதைக் கண்டு வருந்தி, போர் தொடுப்பதைக் கைவிட்டார். <end>

Original [4]: <start> He embraced Buddhism and then devoted his life to spread the message of peace and dharma. <end>
Translated [4]: அதற்குப் பிறகு புத்த சமயத்தைத் தழுவி, அமைதியையும் அறத்தையும் பரப்புவதற்காகத் தன் வாழ்வையே அர்ப்பணித்தார். <end>

Original [5]: <start> His service for the cause of public good was exemplary. <end> <pad>

<start> The most famous ruler of ancient India was Emperor Ashoka. <end> <pad> <pad> <pad> <pad> <pad> <pad>

<start> பண்டைய இந்திய அரசர்களில் பேரும் புகழும் பெற்ற அரசர் அசோகர் ஆவார். <pad> <pad> <pad>

<start> It was during his period that Buddhism spread to different parts of Asia. <end> <pad> <pad> <pad>

<start> இவரது ஆட்சியில் தான் புத்த மதம் ஆசியாவின் புத்த ஆற்றிய அசோகரே பின் <unk> பேரும்

<start> Ashoka gave up war after seeing many people grieving death after the Kalinga war. <end> <pad> <pad>

<start> கலிங்கப் தனியே மடிவதைக் பின் வாழ்வையே தழுவி, பெற்ற அரசர்களில் அரசர்களில் அரசர்களில் அரசர்களில் அரசர்களில்

<start> He embraced Buddhism and then devoted his life to spread the message of peace and dharma. <end>

<start> அதற்குப் பிறகு புத்த சமயத்தைத் தழுவி, அமைதியையும் அறத்தையும் பரப்புவதற்காகத் தன் வாழ்வையே பெற்ற அசோகர்

<start> His service for the cause of public good was exemplary. <end> <pad> <pad> <pad> <pad> <pad> <pad>

<start> பொதுமக்களுக்கு அவர் ஆற்றிய சேவை முன் மாதிரியாக விளங்கியது. <pad> <pad> <pad> <pad> <pad>

<start> He was the first ruler to give up war after victory. <end> <pad> <pad> <pad> <pad> <pad>

<start> வெற்றிக்குப் பின் போரைத் துறந்த முதல் அரசர் அசோகர்தான். <pad> <pad> <pad> <pad> <pad>

<start> He was the first to build hospitals for animals. <end> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

<start> உலகிலேயே முதன்முதலாக விலங்குகளுக்கும் தனியே மருத்துவமனை அமைத்துத் தந்தவரும் அசோகரே ஆவார். <pad> <pad> <pad>

<start> He was the first to lay roads. <end> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

<start>  இன்றும் அவர் உருவாக்கிய சாலைகளை நாம் பயன்படுத்திக்கொண்டு இருக்கிறோம். <pad> <pad> <pad> <pad>